# Building Abstractions with Procedures

## Exercise 1.1
Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented
```racket
10
(+ 5 3 4)
(- 9 1)
(/ 6 2)
(+ (* 2 4) (- 4 6))
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
(= a b)
(if (and (> b a) (< b (* a b)))
    b
    a)
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
(+ 2 (if (> b a) b a))
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
```

In [56]:
10

10

In [57]:
(+ 5 3 4)

12

In [58]:
(- 9 1)

8

In [59]:
(/ 6 2)

3

In [60]:
(+ (* 2 4) (- 4 6))

6

In [61]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))

19

In [62]:
(= a b)

#f

In [63]:
(if (and (> b a) (< b (* a b)))
    b
    a)

4

In [64]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

In [65]:
(+ 2 (if (> b a) b a))

6

In [66]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

## Exercise 1.2
Translate the following expression into prefix form
$$
\frac{5+4+(2-(3-(6+\frac{4}{3})))}{3(6-2)(2-7)}
$$

In [67]:
(/ (+ 5 4 (- 2
             (- 3
                (+ 6 (/ 4 3)))))
   (* 3 (- 6 2) (- 2 7)))

-23/90

## Exercise 1.3
Define a procedure that takes three numbers as arguments and returns the sum of the squares of the two larger numbers.

In [68]:
(define (square x)
  (* x x))
(define (sum_of_squares a b)
  (+ (square a) (square b)))
(define (sum_square_of_two_larger x y z)
  (cond ((and (> x z) (> y z))
        (sum_of_squares x y))
        ((> x y)
         (sum_of_squares x z))
        (else (sum_of_squares y z))))